In [ ]:
# For SOC Feasibility - We want to understand battery charge percentage so when we need to charge it what the best time to discharge it

## Points to be taken:
1. We can assume that energy loss is zero
2. Amount taken to charge or discharge is 2 Hours
3. Initally SOC% can be 0 to 100 which will affect what we need first like we have buy or sell power
4. If battery is fully charged or fully depeleted i.e. 100% or 0% then either charge it or discharge it.0
5. Good options - charge two times then sell 2 times - we need to maximise TB2 revenue so have to constanly cheking T2-B2.
6. First we will locate top and bottom 2 values then we will optimise with SOC% and find the best way for max(revenue) in a given day. 
8. In this I am only finding for one day without checking data of next day which can created more variables. 
9. We can also calculated from point 8 but it will be more complicated. We can all arrange all prices and dates in one single string and then calculate a way which
can maximise revenue. In this we will find for 365*24. 
10. In this we can only calulate for 1 hr for which buy at lower price and sell it higher price. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("hourly_prices.csv")
df['date_only'] = df['date'].apply(lambda x: x.split(" ")[0])
df['time'] = df['date'].apply(lambda x: x.split(" ")[1])
df

,date,price,date_only,time
0,2022-01-01 00:00:00,64.36,2022-01-01,00:00:00
1,2022-01-01 01:00:00,32.45,2022-01-01,01:00:00
2,2022-01-01 02:00:00,25.54,2022-01-01,02:00:00
3,2022-01-01 03:00:00,22.42,2022-01-01,03:00:00
4,2022-01-01 04:00:00,15.95,2022-01-01,04:00:00
...,...,...,...,...
8755,2022-12-31 19:00:00,9.21,2022-12-31,19:00:00
8756,2022-12-31 20:00:00,7.76,2022-12-31,20:00:00
8757,2022-12-31 21:00:00,2.45,2022-12-31,21:00:00
8758,2022-12-31 22:00:00,-1.64,2022-12-31,22:00:00


## Actions for this problem - Testing to Solution
1. First finding 4-5 top and bottom values
2. Then sum of 4 values (2 values each time) also their index in both cases top and bottom separetely
3. and compared then with given that index sum of bottom is lower then top index sum
4. Find maximum T2-B2
append that in list
add that in dataframe with unique date

In [3]:
j=0
uni_date=[]
count=0
max_diff_Rev = []
for i in df['date_only'].unique():
    # This is for how many top and bottoms values we needed. This also gives all combination for both top and bottom values
    top_bottom_val = 24
    
    top_n_price=[]
    top_n_time=[]
    bottom_n_price=[]
    bottom_n_time=[]

    # Unique date list
    uni_date.append(i)

    # Top n prices for each day
    top_prices = (df[df['date_only']==i].nlargest(top_bottom_val,['price']).price)
    top_n_price.append(top_prices.tolist())
    
    # Top n prices time(hour) each day
    top_time = df[df['date_only']==i].nlargest(top_bottom_val,['price']).time.apply(lambda x: x.split(":")[0])
    top_n_time.append(top_time.tolist())

    # Bottom n prices for each day
    bottom_prices = (df[df['date_only']==i].nsmallest(top_bottom_val,['price']).price)
    bottom_n_price.append(bottom_prices.tolist())

    # Bottom n prices time(hour) each day
    bottom_time = df[df['date_only']==i].nsmallest(top_bottom_val,['price']).time.apply(lambda x: x.split(":")[0])
    bottom_n_time.append(bottom_time.tolist())

    # Combination for both top and bottom separately
    from itertools import combinations
    # Generate combinations of length 2 from list
    combs = list(combinations(top_n_price[0], 2))
    # Generate list 'B' by adding each pair of combinations
    B = [round(x + y,2) for x, y in combs]
    #Combination of time which is most important for maintaining SOC feasibility
    combs_tt = list(combinations(top_n_time[0], 2))
    # Generate list 'C' by adding each pair of time(hour)
    C = [int(x) + int(y) for x, y in combs_tt]
    a=[]
    for i in range(len(B)):
        a.append([B[i],C[i]])

    ## For Bottom 
    
    # Generate combinations of length 2 from list
    combso = list(combinations(bottom_n_price[0], 2))
    
    # Generate list 'Bo' by adding each pair of combinations
    Bo = [round(x + y,2) for x, y in combso]
    #Combination of time which is most important for maintaining SOC feasibility
    combso_bt = list(combinations(bottom_n_time[0], 2))
    
    # Generate list 'Co' by adding each pair of combinations
    Co = [int(x) + int(y) for x, y in combso_bt]
   
    ao=[]
    for i in range(len(B)):
        ao.append([Bo[i],Co[i]])

    #Compare max difference between top and bottom where bottom index sum < top index sum
    # Assuming SOC(%)=0 as this is intial phase
    
    def maxRev(a,ao): 
        revm = -float('inf')
        for i in range(len(a)):
            for k in range(len(a)):
                # bottom index < top index
                if(ao[k][1]<a[i][1]):
                    B1 = int(combso_bt[k][0])
                    B2 = int(combso_bt[k][1])
                    T1 = int(combs_tt[i][0])
                    T2 = int(combs_tt[i][1])
                    # SOC is zero, so there is no energy in battery. First we have buy at lower value
                    # From all combination, it is identified for finding TB2, there are only two options avaiable:
                    # BBTT or BTBT
                    # From this we can concluded that firstly we need to charge
                    # In last we have to discharge battery
                    if(
                        (B1<T1 and B1<T2 or B2<T1 and B2<T2) 
                        and
                        (T1>B1 and T1>B2 or T2>B1 and T2>B2)
                    ):
                        # Finding maximum from all combination which satisfy above conditions
                        revm = max(revm,a[i][0]-ao[k][0])
        return round(revm,2)

    # Appending TB2 in the list 
    max_diff_Rev.append(maxRev(a,ao))
    

In [4]:
# For checking if there is, for date not fulfilling the condition. Also ensuring everything right by manually also checked for some dates
chs=0
for i in max_diff_Rev:
    if(i==-float('inf')):
        chs=chs+1
print(chs)

0


In [5]:
# Creating new dataframe for results
df_new = pd.DataFrame()
df_new['date'] = uni_date
df_new['TB2_Revenue ($)'] = max_diff_Rev
df_new

,date,TB2_Revenue ($)
0,2022-01-01,40.80
1,2022-01-02,61.40
2,2022-01-03,36.71
3,2022-01-04,44.25
4,2022-01-05,27.52
...,...,...
360,2022-12-27,57.45
361,2022-12-28,55.00
362,2022-12-29,88.42
363,2022-12-30,33.05


In [10]:
round(sum(df_new['TB2_Revenue ($)']),2)

106650.2

In [11]:
df_new.to_csv("modified_daily_tb2.csv",index=False)
# df_load = pd.read_csv("modified_daily_tb2.csv", index=False)

In [9]:
df_new

,date,TB2_Revenue ($)
0,2022-01-01,40.80
1,2022-01-02,61.40
2,2022-01-03,36.71
3,2022-01-04,44.25
4,2022-01-05,27.52
...,...,...
360,2022-12-27,57.45
361,2022-12-28,55.00
362,2022-12-29,88.42
363,2022-12-30,33.05


In [ ]:
# Most important of this is finding right condition and also put a data structres which is suitable for that problem.